In [ ]:
%cd /Users/santiago/thesis/code

# reload magic
%load_ext autoreload
%autoreload 2

from src import utils, kmeans, dba, nn
import numpy as np
import matplotlib.pyplot as plt
from src.Dataset import Dataset

In [ ]:
ds = Dataset("Beef")

print(ds.train_set[0][:20])
print(ds.test_set[0][:20])


In [ ]:
# DBA sanity check

datasets = utils.get_n_datasets(3, low_k=False)

for i, dataset in enumerate(datasets):
    n = 20
    S = utils.get_class_sequences(n+1, dataset) # get 11 sequences from same (random) class

    mean = dba.dba_mean(S, iterations=5, verbose=True)
    utils.plot_cluster(S, mean, i+1, dataset_name=dataset)
        

In [ ]:
# testing kmeans

dataset = utils.get_n_datasets(1, True)[0]

S, classes = utils.get_all_sequences(dataset)
k = len(classes)
n_seqs = S.shape[0]
seq_len = S.shape[1]

# only perform k-means for short-med length sequences
if seq_len < 500 and n_seqs < 1000:
    print(f'Set "{dataset}"\nClasses: {k}\nSequences: {n_seqs}\nSequence length: {seq_len}')

    clusters_indices, centroids = kmeans.find_k_clusters(S, k, verbose=True)
    clusters_arrays = utils.extract_class_sequences(S, clusters_indices)
    utils.plot_clusters(clusters_arrays, centroids)

In [ ]:
# NN Classification

dataset = utils.get_n_datasets(1)[0]

def get_class_for_sequence(class_list: [[int]], seq_index):
    for class_index, a_class in enumerate(class_list):
        if seq_index in a_class: 
            return class_index

# dataset = "CBF"

# Get TRAIN sequences and calculate centroids with DBA
S_train, train_classes = utils.get_all_sequences(dataset)
S_train_list = utils.extract_class_sequences(S_train, train_classes)

class_means = []
for S_c in S_train_list:
    mean = dba.dba_mean(S_c)
    class_means.append(mean)

info = utils.get_dataset_info(dataset)
print(info)


# get TEST classes and classify with nearest neighbor method
S_test, test_classes = utils.get_all_sequences(dataset, train=False)
print(f"Performing NN on Test set of length {S_test.shape[0]}")

calculated_classes = nn.classify(class_means, S_test)  # do the meat

S_test_list = utils.extract_class_sequences(S_test, calculated_classes)
# utils.plot_clusters(S_test_list, class_means)

success = nn.calculate_success_rate(test_classes, calculated_class, S_test.shape[0])

# correctly_classified = 0

# for test_i in range(S_test.shape[0]):
#     labeled_class = get_class_for_sequence(test_classes, test_i)
#     calculated_class = get_class_for_sequence(calculated_classes, test_i)
#     if labeled_class == calculated_class:
#         correctly_classified += 1
    
# print(correctly_classified/S_test.shape[0])

